In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('fivethirtyeight')

In [6]:
#pjme = pd.read_csv('./datalab/62612/中国疫情.csv', index_col=[0], parse_dates=[0])
#pjme = pd.read_csv('./datalab/62616/韩国疫情.csv', index_col=[0], parse_dates=[0])
pjme = pd.read_csv('./datalab/62626/美国test.csv', index_col=[0], parse_dates=[0])
#pjme = pjme.drop(columns = ['Confirmed'])
split_date = '2020/04/20'
pjme_train = pjme.loc[pjme.index <= split_date].copy()
pjme_test = pjme.loc[pjme.index > split_date].copy()
#import numpy as np
#dataset = pd.read_csv('./datalab/62612/中国疫情2.csv')
#col = dataset.columns.values.tolist()  # 获得列的名称
#data = np.array(dataset[col[1:]])  # 取出某一列
#time = np.array(dataset[col[0:1]])
#print(data)
print(pjme)


            Confirmed    New
Date                        
2020-04-01     213242  25070
2020-04-02     243622  30380
2020-04-03     275367  31745
2020-04-04     308650  33283
2020-04-05     336802  28152
2020-04-06     366317  29515
2020-04-07     397121  30804
2020-04-08     428654  31533
2020-04-09     462780  34126
2020-04-10     496535  33755
2020-04-11     526396  29861
2020-04-12     555313  28917
2020-04-13     580619  25306
2020-04-14     607670  27051
2020-04-15     636350  28680
2020-04-16     667592  31242
2020-04-17     699706  32114
2020-04-18     727959  28253
2020-04-19     754376  26417
2020-04-20     779682  25306
2020-04-21     807023  27341
2020-04-22     835150  28127
2020-04-23     869170  34020
2020-04-24     905358  36188
2020-04-25     938154  32796
2020-04-26     965785  27631
2020-04-27     988197  22412
2020-04-28    1012582  24385
2020-04-29    1039909  27327


In [13]:
#from sklearn.model_selection import train_test_split # 载入数据分割函数train_test_split
#X_train,X_test,y_train,y_test = train_test_split(data, # 特征空间
#time, # 输出空间
#test_size = 0.3, # 测试集占30%
#random_state = 0) # 为了复现实验，设置一个随机数
#print(X_train)

In [14]:
#pjme = pd.read_csv('./datalab/62612/中国情况.csv', index_col=[0], parse_dates=[0])
#pjme = pd.read_csv('./datalab/62616/韩国情况.csv', index_col=[0], parse_dates=[0])
#pjme = pd.read_csv('./datalab/62626/美国情况.csv', index_col=[0], parse_dates=[0])
#split_date = '2020/05/01'
#pjme_train = pjme.loc[pjme.index <= split_date].copy()
#pjme_predict = pjme.loc[pjme.index > split_date].copy()

In [7]:
def create_features(df, label=None):
    df['date'] = df.index # index: DatetimeIndex
    #df['hour'] = df['date'].dt.hour # dt: DatetimeProperties, hour: Series
    #df['day_of_week'] = df['date'].dt.dayofweek
    #df['quarter'] = df['date'].dt.quarter
    #df['month'] = df['date'].dt.month
    #df['year'] = df['date'].dt.year
    df['day_of_year'] = df['date'].dt.dayofyear
    #df['day_of_month'] = df['date'].dt.day
    #df['week_of_year'] = df['date'].dt.weekofyear

    X = df[['day_of_year']]
    if label:
        y = df[label]
        return X, y
    return X

# 训练集
X_train, y_train = create_features(pjme_train, label='New')
# 测试集
X_test, y_test = create_features(pjme_test, label='New')



In [8]:
# 模型
param = {'max_depth':5, 'eta':0.5, 'verbosity':1,'xgb_model':'reg:logistics'}
reg = xgb.XGBRegressor(n_estimators=1000,objective = 'reg:gamma')
# 训练
reg.fit(X_train, y_train,
        eval_set=[ (X_test, y_test)],
        verbose=True,
        early_stopping_rounds=80)
        
     #   eval_metric='logloss',, (X_test, y_test)(X_train, y_train),

[0]	validation_0-gamma-nloglik:52324.6
Will train until validation_0-gamma-nloglik hasn't improved in 80 rounds.
[1]	validation_0-gamma-nloglik:47345.4
[2]	validation_0-gamma-nloglik:42840.1
[3]	validation_0-gamma-nloglik:38763.4
[4]	validation_0-gamma-nloglik:35074.8
[5]	validation_0-gamma-nloglik:31737.1
[6]	validation_0-gamma-nloglik:28717.1
[7]	validation_0-gamma-nloglik:25984.5
[8]	validation_0-gamma-nloglik:23512
[9]	validation_0-gamma-nloglik:21274.7
[10]	validation_0-gamma-nloglik:19250.4
[11]	validation_0-gamma-nloglik:17418.7
[12]	validation_0-gamma-nloglik:15761.3
[13]	validation_0-gamma-nloglik:14261.7
[14]	validation_0-gamma-nloglik:12904.8
[15]	validation_0-gamma-nloglik:11677
[16]	validation_0-gamma-nloglik:10566.1
[17]	validation_0-gamma-nloglik:9560.86
[18]	validation_0-gamma-nloglik:8651.32
[19]	validation_0-gamma-nloglik:7828.35
[20]	validation_0-gamma-nloglik:7083.7
[21]	validation_0-gamma-nloglik:6409.92
[22]	validation_0-gamma-nloglik:5800.28
[23]	validation_0-gam

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:gamma', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [9]:
#plot_importance(reg, height=0.9)
pjme_train['Prediction']  = reg.predict(X_train)
print(pjme_train)

            Confirmed    New       date  day_of_year    Prediction
Date                                                              
2020-04-01     213242  25070 2020-04-01           92  25200.683594
2020-04-02     243622  30380 2020-04-02           93  30254.955078
2020-04-03     275367  31745 2020-04-03           94  31710.972656
2020-04-04     308650  33283 2020-04-04           95  33001.375000
2020-04-05     336802  28152 2020-04-05           96  28603.976562
2020-04-06     366317  29515 2020-04-06           97  29256.787109
2020-04-07     397121  30804 2020-04-07           98  30877.876953
2020-04-08     428654  31533 2020-04-08           99  31609.521484
2020-04-09     462780  34126 2020-04-09          100  33815.312500
2020-04-10     496535  33755 2020-04-10          101  33801.703125
2020-04-11     526396  29861 2020-04-11          102  29948.535156
2020-04-12     555313  28917 2020-04-12          103  28656.291016
2020-04-13     580619  25306 2020-04-13          104  25721.16

In [15]:
# 预测
#pjme_past = pd.concat([pjme_train,pjme_test])
#X_past = pd.concat([X_train,X_test])
X_all = pd.concat([X_train,X_test])
pjme_all = pd.concat([pjme_train,pjme_test])
#print(pjme_test['MW_Prediction'])
#pjme_predict['MW_Prediction'] = reg.predict(X_predict)
#pjme_past['Prediction'] = reg.predict(X_past)
pjme_test['Prediction'] = reg.predict(X_test)
#pjme_train['Prediction'] = reg.predict(X_train)
pjme_all['Prediction'] = reg.predict(X_all)
print(pjme_all)
#print(pjme_train)
print(pjme_test)
#print(pjme_past)


            Confirmed    New       date  day_of_year    Prediction
Date                                                              
2020-04-01     213242  25070 2020-04-01           92  25200.683594
2020-04-02     243622  30380 2020-04-02           93  30254.955078
2020-04-03     275367  31745 2020-04-03           94  31710.972656
2020-04-04     308650  33283 2020-04-04           95  33001.375000
2020-04-05     336802  28152 2020-04-05           96  28603.976562
2020-04-06     366317  29515 2020-04-06           97  29256.787109
2020-04-07     397121  30804 2020-04-07           98  30877.876953
2020-04-08     428654  31533 2020-04-08           99  31609.521484
2020-04-09     462780  34126 2020-04-09          100  33815.312500
2020-04-10     496535  33755 2020-04-10          101  33801.703125
2020-04-11     526396  29861 2020-04-11          102  29948.535156
2020-04-12     555313  28917 2020-04-12          103  28656.291016
2020-04-13     580619  25306 2020-04-13          104  25721.16

In [13]:

pjme_all[['New','Prediction']].plot(figsize=(15, 5))

NameError: name 'pjme_all' is not defined

In [12]:
import matplotlib.pyplot as plt
# 画出预测图
plot1 = plt.plot(pjme_all.index, pjme_all['Confirmed'], 's',label="已确诊人数")
plot2 = plt.plot(pjme_all.index, pjme_all['MW_Prediction'], 's',label='预测的病例数')
plot3 = plt.plot(pjme_all.index, pjme_all['Prediction'], 'r',label='预测曲线')
plt.xlabel('天数')
plt.ylabel('感染人群数目')
plt.legend(loc=0)

NameError: name 'pjme_all' is not defined